In [38]:
example = """
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
"""[1:-1].splitlines()

with open("day21.txt", "r") as f:
    data = f.readlines()

In [39]:
from typing import Set

class Food(object):
    def __init__(self, ingredients: Set[str], allergens: Set[str]):
        self.ingredients = set(ingredient.strip() for ingredient in ingredients)
        self.allergens = set(allergens)

    def __str__(self) -> str:
        return f"{' '.join(self.ingredients)} (contains {', '.join(self.allergens)})"

    @staticmethod
    def from_str(input: str) -> "Food":
        ingredients, allergens = input.strip()[:-1].split('(contains ')
        return Food(set(ingredient.strip() for ingredient in ingredients.split(' ') if ingredient), set(allergen.strip() for allergen in allergens.split(', ') if allergen))

print(Food.from_str("sqjhc fvjkl (contains soy)\r"))
assert Food.from_str("sqjhc fvjkl (contains soy)\r").ingredients == set(["sqjhc", "fvjkl"])
assert Food.from_str("sqjhc fvjkl (contains soy)\r").allergens == set(["soy"])

example_foods = list(map(Food.from_str, example))
true_foods = list(map(Food.from_str, data))

sqjhc fvjkl (contains soy)


In [42]:
from typing import Iterable
from collections import defaultdict

class AllergenMap(object):
    def __init__(self, foods: Iterable[Food]):
        foods_containing_allergens = defaultdict(lambda: [])
        allergen_names = {}
        self.known_ingredients = set()

        for food in foods:
            for allergen in food.allergens:
                foods_containing_allergens[allergen].append(food)

            self.known_ingredients.update(food.ingredients)

        have_unresolved_ingredients = True
        while have_unresolved_ingredients:
            have_unresolved_ingredients = False
            for allergen, foods in foods_containing_allergens.items():
                if allergen in allergen_names:
                    continue

                common_ingredients = foods[0].ingredients - set(allergen_names.values())
                for food in foods:
                    common_ingredients = common_ingredients.intersection(food.ingredients)

                if len(common_ingredients) == 0:
                    raise Exception(f"Could not resolve common ingredient name for {allergen}")
                elif len(common_ingredients) == 1:
                    ingredient = next(iter(common_ingredients))
                    allergen_names[allergen] = ingredient
                else:
                    have_unresolved_ingredients = True

        self.allergens = allergen_names
        self.safe_ingredients = self.known_ingredients - set(self.allergens.values())

    def dangerous_ingredients(self) -> Iterable[str]:
        return [
            self.allergens[key]
            for key in sorted(self.allergens.keys())
        ]

example_map = AllergenMap(example_foods)
print(f"Example Safe Ingredient Appearances (Part 1): {sum(1 for food in example_foods for ingredient in food.ingredients if ingredient in example_map.safe_ingredients)}")
print(f"Example Dangerous Ingredients (Part 2): {','.join(example_map.dangerous_ingredients())}")

true_map = AllergenMap(true_foods)
print(f"True Safe Ingredient Appearances (Part 1): {sum(1 for food in true_foods for ingredient in food.ingredients if ingredient in true_map.safe_ingredients)}")
print(f"True Dangerous Ingredients (Part 2): {','.join(true_map.dangerous_ingredients())}")

Example Safe Ingredient Appearances (Part 1): 5
Example Dangerous Ingredients (Part 2): mxmxvkd,sqjhc,fvjkl
True Safe Ingredient Appearances (Part 1): 2020
True Dangerous Ingredients (Part 2): bcdgf,xhrdsl,vndrb,dhbxtb,lbnmsr,scxxn,bvcrrfbr,xcgtv
